# Lab #1

---

## Task
Для СЛАУ с некоторой матрицей A:

- вычислить числа обусловленности
- поварьировав матрицу и правую часть, вычислить |x - ͂x͂|
- посмотреть, есть ли корреляция между величинами чисел обусловленности и погрешностью решения

## Solution
---

### Computation of condition numbers

In [23]:
from scipy import linalg as la
import numpy as np
import math

def spectral_condition_nums(matr):
    return la.norm(matr) * la.norm(la.inv(matr))

def bulk_condition_nums(matr):
    product = 1
    n, m = matr.shape
    
    for i in range(n):
        temp_sum = 0
        for j in range(m):
            temp_sum += pow(matr[i, j], 2)
        product *= math.sqrt(temp_sum)
    
    return product / abs(la.det(matr))

def angular_condition_nums(matr):
    iterator = zip(matr, la.inv(matr).transpose())
    return max([la.norm(row) * la.norm(col) for (row, col) in iterator])

def compute_matr_condition_nums(matr):
    return [
        spectral_condition_nums(matr),
        bulk_condition_nums(matr),
        angular_condition_nums(matr)
    ]

### Linear equation solution

In [24]:
def solve_le(A, b):
    return la.solve(A, b)

### Creation of test matrices

In [25]:
def round_matrices(matrices, decimals):
    return [matrix.round(decimals) for matrix in matrices]

def create_hilbert_matrices(st_i, end_i):
    return [la.hilbert(i) for i in range(st_i, end_i)]

def create_tridiagonal_matrix(size):
    if size > 2:
        zeros = [0 for i in range(size - 2)]
    else:
        raise AttributeError('size should be greater than 2.')
    return np.array(la.toeplitz([2, -1] + zeros, [0, -1] + zeros))

def create_tridiagonal_matrices(st_i, end_i):
    return [create_tridiagonal_matrix(i) for i in range(st_i, end_i)]

def create_random_matrices(st_i, end_i):
    return [np.random.random((i, i)) for i in range(st_i, end_i)]

## Experimental research

---

This block presents results of the implemented functions.
We evaluate three different test cases:
- Hilbert matrices
- Randomly generated matrices
- Diagonally dominant tridiagonal matrices
    
For each case we vary matrices and vectors of linear system to check the correlation between
condition numbers and solution error.

In [26]:
import pandas as pd

order_from, order_to = 3, 20
decimals = range(2, 11, 3)

hilbert_matrices = create_hilbert_matrices(order_from, order_to)
tridiagonal_matrices = create_tridiagonal_matrices(order_from, order_to)
random_matrices = create_random_matrices(order_from, order_to)

def experiment(matr, decimal_round):
    n, m = matr.shape
    id_vect = np.ones(m)
    
    b = matr.dot(id_vect)
    b = b.round(decimal_round)
    matr = matr.round(decimal_round)
    
    sol = solve_le(matr, b)
    
    return la.norm(sol - id_vect)

### Results for Hilbert matrices

In [27]:
res_hilbert = []
cond_nums = []

for matr in hilbert_matrices:
    hilbert_results = [experiment(matr, decimal) for decimal in decimals]
    res_hilbert.append(hilbert_results)
    cond_nums.append(compute_matr_condition_nums(matr))

df_h = pd.DataFrame(res_hilbert, index=range(order_from, order_to), columns=decimals)
df_hcn = pd.DataFrame(cond_nums, index=range(order_from, order_to), columns=["spectral", "bulk", "angular"])

df_h = df_h.round(decimals=6)
df_hcn = df_hcn.round(decimals=3)

print(pd.concat([df_h, df_hcn], axis=1))

            2         5         8      spectral           bulk       angular
3    0.000000  0.000000  0.000000  5.261590e+02   7.580470e+02  1.728870e+02
4    0.000000  0.054608  0.000000  1.561379e+04   9.370859e+05  4.020913e+03
5    1.309666  1.339712  0.000000  4.808491e+05   1.608573e+10  9.515770e+04
6    1.624585  9.614305  0.000000  1.511899e+07   3.955105e+15  2.441571e+06
7    0.307761  5.575368  1.151815  4.817473e+08   1.421000e+22  7.162186e+07
8    2.883488  5.437613  2.643599  1.549362e+10   7.564448e+29  2.025215e+09
9    5.412924  2.901804  0.000000  5.017293e+11   6.027526e+38  5.577427e+10
10   3.741657  2.652971  1.108988  1.633184e+13   7.244802e+48  1.719678e+12
11   9.985970  0.000000  0.704408  5.326132e+14   1.318724e+60  5.223034e+13
12  15.611931  0.000000  0.893006  1.668630e+16   3.505211e+72  1.475688e+15
13  11.110376  2.548140  1.420309  3.704761e+17   1.008812e+86  3.021442e+16
14   2.775821  2.277786  1.132922  7.180739e+17   3.416110e+99  5.462015e+16

### Results for randomly generated matrices

In [28]:
res_random = []
cond_nums = []

for matr in random_matrices:
    random_matrices_results = [experiment(matr, decimal) for decimal in decimals]
    res_random.append(random_matrices_results)
    cond_nums.append(compute_matr_condition_nums(matr))

df_r = pd.DataFrame(res_random, index=range(order_from, order_to), columns=decimals)
df_rcn = pd.DataFrame(cond_nums, index=range(order_from, order_to), columns=["spectral", "bulk", "angular"])

df_r = df_r.round(decimals=6)
df_rcn = df_rcn.round(decimals=3)

print(pd.concat([df_r, df_rcn], axis=1))

            2         5         8  spectral          bulk   angular
3    0.031021  0.000031  0.000000    11.460  8.853000e+00     4.902
4    0.077238  0.000350  0.000000   116.528  4.501680e+02    51.042
5    0.049145  0.000069  0.000000    29.576  3.542000e+01     9.477
6    0.031944  0.000090  0.000000    24.040  2.002460e+02     4.822
7    0.252977  0.000100  0.000000   139.645  7.630210e+02    25.691
8    0.069144  0.000036  0.000000    42.235  6.426430e+02    11.270
9    0.121662  0.000077  0.000000    92.608  4.969709e+03    14.803
10   0.198090  0.000253  0.000000    92.145  4.205197e+04    15.627
11   5.609437  0.005867  0.000006  9214.970  8.025473e+06  1842.742
12   0.385585  0.001166  0.000000   355.179  1.949785e+05    55.645
13   0.064982  0.000186  0.000000   164.394  2.255154e+06    23.068
14   0.088908  0.000431  0.000000  1171.806  3.703155e+07   185.615
15   0.200606  0.000084  0.000000   166.013  5.309414e+06    25.524
16   0.348994  0.001157  0.000001   682.324  1.2

### Result for diagonally dominant tridiagonal matrices

In [29]:
res_tridiag = []
cond_nums = []

for matr in tridiagonal_matrices:
    tridiag_results = [experiment(matr, decimal) for decimal in decimals]
    res_tridiag.append(tridiag_results)
    cond_nums.append(compute_matr_condition_nums(matr))

df = pd.DataFrame(res_tridiag, index=range(order_from, order_to), columns=decimals)
df1 = pd.DataFrame(cond_nums, index=range(order_from, order_to), columns=["spectral", "bulk", "angular"])

df = df.round(decimals=9)
df1 = df1.round(decimals=3)

print(pd.concat([df, df1], axis=1))

      2    5    8  spectral         bulk  angular
3   0.0  0.0  0.0     7.211        3.062    3.000
4   0.0  0.0  0.0    12.931        6.000    3.950
5   0.0  0.0  0.0    20.739       12.247    5.339
6   0.0  0.0  0.0    30.854       25.714    6.547
7   0.0  0.0  0.0    43.474       55.114    8.124
8   0.0  0.0  0.0    58.780      120.000    9.545
9   0.0  0.0  0.0    76.943      264.545   11.292
10  0.0  0.0  0.0    98.122      589.091   12.898
11  0.0  0.0  0.0   122.471     1322.724   14.799
12  0.0  0.0  0.0   150.133     2990.769   16.572
13  0.0  0.0  0.0   181.248     6802.583   18.615
14  0.0  0.0  0.0   215.950    15552.000   20.539
15  0.0  0.0  0.0   254.366    35713.560   22.716
16  0.0  0.0  0.0   296.621    82334.118   24.781
17  0.0  0.0  0.0   342.835   190472.322   27.083
18  0.0  0.0  0.0   393.126   442004.211   29.281
19  0.0  0.0  0.0   447.606  1028550.541   31.702
